In [21]:
import gzip
import pandas as pd
import re

# Getting Data

Using sample info from SRA to rename samples:

In [3]:
%%bash 
cp ./methylation_fastq/SRR3083926_1.chr6.fastq ./methylation_fastq/E4.0ICM_rep1.chr6.fastq
cp ./methylation_fastq/SRR3083926_2.chr6.fastq ./methylation_fastq/E4.0ICM_rep2.chr6.fastq
cp ./methylation_fastq/SRR3083929_1.chr6.fastq ./methylation_fastq/E5.5Epi_rep1.chr6.fastq
cp ./methylation_fastq/SRR3083929_2.chr6.fastq ./methylation_fastq/E5.5Epi_rep2.chr6.fastq

Generating fastqc report:

In [ ]:
%%bash
fastqc  SRR3083926_1.chr6.fastq 

Output:
    SRR3083926_1.chr6_fastqc.zip
    SRR3083926_1.chr6_fastqc.html

Looking at the fastqc output, the only thing that looks strange about the data is that the lack of Cs (and, correspondingly, the large number of Ts). This is expected in bisulfite sequencing.

# Bisulfite mapping with Bismark

Chr6.fa was moved into a directory called 'mm10'. The following commands were run from the methylation_fastq directory. Indexing through Bismark was performed as such: 

In [ ]:
%%bash
bismark_genome_preparation ./mm10

This generates the Bisulfite_Genome directory within mm10. 

Running bismark on pairs of replicates:

In [ ]:
%%bash
bismark --fastq ./mm10 -1 E4.0ICM_rep1.chr6.fastq,E5.5Epi_rep1.chr6.fastq -2 E4.0ICM_rep2.chr6.fastq,E5.5Epi_rep2.chr6.fastq

This generates the following pairs of files:

    E4.0ICM_rep1.chr6_bismark_bt2_PE_report.txt
    E4.0ICM_rep1.chr6_bismark_bt2_pe.bam

The same files are made for E5.5

The report.txt file contains info about the number of sequences, mapping efficiency, the number of methylated Cs, etc.
The bam file is used for subsequent analysis

Sorting for IGV visualization: 

In [ ]:
%%bash 
samtools sort E4.0ICM_rep1.chr6_bismark_bt2_pe.bam > E4.0ICM_rep1.chr6_bismark_bt2_pe.sorted.bam
samtools sort E5.5Epi_rep1.chr6_bismark_bt2_pe.bam > E5.5Epi_rep1.chr6_bismark_bt2_pe.sorted.bam

Indexing:

In [ ]:
%%bash
samtools index E4.0ICM_rep1.chr6_bismark_bt2_pe.sorted.bam
samtools index E5.5Epi_rep1.chr6_bismark_bt2_pe.sorted.bam

This generates .bam.bai files with, named as: 

    E4.0ICM_rep1.chr6_bismark_bt2_pe.sorted.bam.bai
    
When running IGV to visualize, use fname.sorted.bam as input. The fname.bam.bai files are found and used automatically.

Methylation data is extracted as such:

In [ ]:
%%bash
bismark_methylation_extractor --bedgraph --comprehensive E4.0ICM_rep1.chr6_bismark_bt2_pe.sorted.bam E5.5Epi_rep1.chr6_bismark_bt2_pe.sorted.bam

This gives a ton of output files. Specifically, they are of the format: 
    CHG_context_fname.txt
    CHH_context_fname.txt
    CpG_context_fname.txt
    fname_splitting_report.txt
    fname.bedGraph.gz
    fname.bismark.cov.gz
    fname.M-bias.txt

We will use the fname.bedGraph.gz file as input for IGV.

# Python

In [75]:
gene_coordinates = pd.read_csv("methylation_fastq/mm10/mm10_refseq_genes_chr6_50M_60M.bed", sep='\t', header=None)
#gene_coordinates

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,967,NM_001164734,chr6,+,50110240,50198598,50145775,50198330,14,"50110240,50143718,50145773,50158681,50162585,5...","50110464,50143784,50145892,50158834,50162735,5...",0,Mpp6,cmpl,cmpl,"-1,-1,0,0,0,0,0,0,0,1,1,1,0,0,"
1,967,NM_001164733,chr6,+,50110240,50198598,50145775,50198330,14,"50110240,50143718,50145773,50158681,50162585,5...","50110454,50143784,50145892,50158834,50162735,5...",0,Mpp6,cmpl,cmpl,"-1,-1,0,0,0,0,0,0,0,1,1,1,0,0,"
2,967,NM_019939,chr6,+,50110240,50198598,50145775,50198330,13,"50110240,50143718,50145773,50158681,50162585,5...","50110464,50143784,50145892,50158834,50162735,5...",0,Mpp6,cmpl,cmpl,"-1,-1,0,0,0,0,0,0,1,1,1,0,0,"
3,968,NM_018769,chr6,-,50207402,50261769,50207933,50251486,10,"50207402,50216349,50219236,50220988,50222735,5...","50208206,50216423,50219438,50221116,50222900,5...",0,Dfna5,cmpl,cmpl,"0,1,0,1,1,0,2,1,0,-1,"
4,1,NM_027881,chr6,-,50293326,50382837,50297064,50370215,22,"50293326,50299315,50300923,50303016,50308290,5...","50297161,50299438,50301050,50303161,50308435,5...",0,Osbpl3,cmpl,cmpl,"2,2,1,0,2,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,-1,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,129,NM_028705,chr6,+,58833699,58920396,58843632,58918921,25,"58833699,58843603,58854872,58855737,58856592,5...","58833771,58843858,58855032,58855814,58856814,5...",0,Herc3,cmpl,cmpl,"-1,0,1,2,1,1,0,2,1,0,2,2,0,1,0,0,0,0,0,0,2,0,2..."
148,1034,NM_021432,chr6,-,58905232,58907126,58906496,58906967,1,"58905232,","58907126,",0,Nap1l5,cmpl,cmpl,"0,"
149,129,NM_153574,chr6,-,58933535,59024502,58935584,59024340,18,"58933535,58939295,58940120,58944120,58946039,5...","58935711,58939397,58940317,58944204,58946135,5...",0,Fam13a,cmpl,cmpl,"2,2,0,0,0,0,1,0,2,1,0,2,0,2,2,2,2,0,"
150,1036,NM_001081145,chr6,+,59208869,59212033,59210149,59211727,2,"59208869,59209338,","59209181,59212033,",0,Tigd2,cmpl,cmpl,"-1,0,"


13th column is gene name – filter unique genes. Columns 5 and 6 are position start and end

Reading in bedGraph.gz files:

In [117]:
with gzip.open("methylation_fastq/E4.0ICM_rep1.chr6_bismark_bt2_pe.sorted.bedGraph.gz", 'r') as f:
    E4 = f.readlines()

E4 = E4[1::]

In [132]:
with gzip.open("methylation_fastq/E5.5Epi_rep1.chr6_bismark_bt2_pe.sorted.bedGraph.gz", 'r') as f:
    E5 = f.readlines()

E5 = E5[1::]

In [70]:
print(str(E4[999], 'utf-8').split())
print(str(E4[1000], 'utf-8').split())
print(str(E4[1001], 'utf-8').split())

['chr6', '50186852', '50186853', '0']
['chr6', '50186863', '50186864', '20']
['chr6', '50186999', '50187000', '0']


In [96]:
type(str(E4[1000], 'utf-8').split()[1])

str

In [147]:
# Making a condensed list which only includes gene name, start, and stop. 
# This list only includes unique genes
condensed_list = []
for gene in gene_coordinates.iterrows():
    if [gene[1][12], gene[1][4], gene[1][5]] not in condensed_list:
        condensed_list.append([gene[1][12], gene[1][4], gene[1][5]])

In [150]:
condensed_list[:10]

[['Mpp6', 50110240, 50198598],
 ['Dfna5', 50207402, 50261769],
 ['Osbpl3', 50293326, 50382837],
 ['Osbpl3', 50293326, 50456170],
 ['Cycs', 50562562, 50566474],
 ['5430402O13Rik', 50566642, 50594865],
 ['Mir6371', 50572397, 50572508],
 ['4921507P07Rik', 50573303, 50596590],
 ['Npvf', 50650870, 50654393],
 ['C530044C16Rik', 50776114, 50814894]]

Calculating average methylation per gene

In [137]:
E4_averages = {}
for gene in condensed_list:
    E4_total_signal=0
    E4_num_signals=0
    for l in E4:
        line=str(l, 'utf-8').split()
        if line[-1] != '0':
            if (int(line[1]) > int(gene[1]) and int(line[1]) < int(gene[2])) or (int(line[2]) > int(gene[1]) and int(line[2]) < int(gene[2])):
                E4_total_signal+=float(line[3])
                E4_num_signals+=1
                #print(total_signal)
                #print(num_signals)
                #print(gene)
                #print(line)
    if E4_num_signals > 0:
        E4_avg = E4_total_signal / E4_num_signals
        E4_averages[gene[0]] = E4_avg

#E4_averages

E5_averages = {}
for gene in condensed_list:
    E5_total_signal=0
    E5_num_signals=0
    for l in E5:
        line=str(l, 'utf-8').split()
        if line[-1] != '0':
            if (int(line[1]) > int(gene[1]) and int(line[1]) < int(gene[2])) or (int(line[2]) > int(gene[1]) and int(line[2]) < int(gene[2])):
                E5_total_signal+=float(line[3])
                E5_num_signals+=1
                #print(total_signal)
                #print(num_signals)
                #print(gene)
                #print(line)
    if E5_num_signals > 0:
        E5_avg = E5_total_signal / E5_num_signals
        E5_averages[gene[0]] = E5_avg

In [151]:
ratios = []
for gene in E4_averages.keys():
    if gene in E5_averages.keys():
        ratios.append([gene, float(E4_averages[gene])/float(E5_averages[gene])])
#ratios

In [152]:
with open('E4-E5_methylation_ratios.txt', 'w') as f:
    for gene in ratios:
        f.write("%s\n" % gene)